In [ ]:
import os

os.environ['KAGGLE_USERNAME'] = "daxinniu122" # username from the json file
os.environ['KAGGLE_KEY'] = "2ed6804df3e3e2e0d5053c862fa7020a" # key from the json file

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import zipfile
for i in os.listdir('/content/'):
    if 'zip' in i:
        with zipfile.ZipFile(i, 'r') as f:
            f.extractall('/content/drive/MyDrive/cassava-leaf-disease-classification/')

In [2]:
# Library import
import pandas as pd
import os
import re
import tensorflow as tf
import pandas as pd
import os
import re
import cv2
import albumentations as alb
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.applications import VGG19, ResNet152V2, InceptionResNetV2, ResNet50
from tensorflow.keras.layers import AveragePooling2D, Activation, GlobalAveragePooling2D
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from functools import partial
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import Dense, Dropout, Flatten, GlobalAveragePooling2D, BatchNormalization, Activation
from tensorflow.keras.applications import EfficientNetB3, EfficientNetB4
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.applications import VGG16
from functools import partial
from sklearn.model_selection import train_test_split

# Constant Variables:
_auto_tune = tf.data.experimental.AUTOTUNE
_batch_size = 32

_image_width_original = 512
_image_height_original = 512
_image_size = [_image_width_original, _image_height_original]

_image_resize_width = 336
_image_resize_height = 336
_image_resize = [_image_resize_width, _image_resize_height]
print('Model input shape {} x {}.'.format(_image_resize_width, _image_resize_height))

_channels = 3
_n_class = 5
_n_repeat = 4
_img_norm = 255.0

_classes = [str(x) for x in range(_n_class)]
_major_label = 3
_classes_names = ['Cassava Bacterial Blight',
                  'Cassava Brown Streak Disease',
                  'Cassava Green Mottle',
                  'Cassava Mosaic Disease',
                  'Healthy']
_train_file = '/content/drive/MyDrive/cassava-leaf-disease-classification/'
_train_recs = list(filter(lambda x: '.tfrec' in x, os.listdir(_train_file)))
_epochs = 20
_valid_size = 0.1
_train_df = pd.read_csv('/content/drive/MyDrive/cassava-leaf-disease-classification/train.csv', encoding='utf_8_sig',
                        engine='python')
_file_label_map = dict(zip(_train_df.image_id.tolist(), _train_df.label.astype(int).tolist()))
_random_corp_size = [_image_resize_width, _image_resize_height, _channels]


# Decoding single image:
def decode_img(img,
               n_channels: int = _channels,
               img_size: list = None,
               img_norm: float = _img_norm):
    if img_size is None:
        img_size = _image_size
    img = tf.image.decode_jpeg(img, channels=n_channels)
    img = tf.reshape(img, [*img_size, n_channels])
    return img


# Parsing the files
def parse_img(x,
              n_class: int = _n_class):
    feature_description = {'image': tf.io.FixedLenFeature([], tf.string, default_value=''),
                           'target': tf.io.FixedLenFeature([], tf.int64, default_value=-1)}
    parsed_features = tf.io.parse_single_example(x, feature_description)
    img = decode_img(parsed_features['image'])
    label = tf.one_hot(parsed_features['target'], depth=n_class)
    return img, label


# Load data
def load_img(files: list,
             ordered=False):
    df = tf.data.TFRecordDataset(files)
    ignore_order = tf.data.Options()
    if not ordered:
        ignore_order.experimental_deterministic = False
    df = df.with_options(ignore_order)
    df = df.map(parse_img)
    return df


# Train-validation split
_train_fn, _valid_fn = \
    train_test_split(tf.io.gfile.glob(_train_file + 'ld_train*.tfrec'),
                     test_size=_valid_size,
                     random_state=5,
                     shuffle=True)

Model input shape 336 x 336.


In [3]:

# Function for getting the training data set
def get_train_data(train_fn: list = _train_fn,
                   batch_size: int = _batch_size):
    df = load_img(train_fn)
    df = df.repeat().shuffle(2048).batch(batch_size).prefetch(_auto_tune)
    return df


# Function for getting the validation data set
def get_valid_data(valid_fn: list = _valid_fn,
                   batch_size: int = _batch_size):
    df = load_img(valid_fn)
    df = df.batch(batch_size).cache().prefetch(_auto_tune)
    return df


# Reporting the size of training, validation and testing data
def report_data_size(train_f=_train_fn,
                     valid_f=_valid_fn):
    def count_file(x):
        return sum([int(re.compile(r"-([0-9]*)\.").search(i).group(1)) for i in x])

    n_train, n_valid= count_file(train_f), count_file(valid_f)
    print('Train Images: {} | Validation Images: {}'.format(n_train, n_valid))
    return n_train, n_valid


# Check the size of the data
_n_train, _n_valid = report_data_size()

# Fetching training, validation and testing data
train_data = get_train_data()
valid_data = get_valid_data()

print("Train Data Size {} | Validation Data Size {}".format(train_data._flat_shapes, valid_data._flat_shapes))

Train Images: 18721 | Validation Images: 2676
Train Data Size [TensorShape([None, 512, 512, 3]), TensorShape([None, 5])] | Validation Data Size [TensorShape([None, 512, 512, 3]), TensorShape([None, 5])]


In [4]:
input_module = tf.keras.layers.Input(shape=(*_image_size, _channels))
flip_module = tf.keras.layers.experimental.preprocessing.RandomFlip(mode='horizontal_and_vertical')
roration_module = tf.keras.layers.experimental.preprocessing.RandomRotation(factor=(-0.4, 0.4))
crop_module = tf.keras.layers.experimental.preprocessing.RandomCrop(height=_image_resize_height, width=_image_resize_width)
rand_height_module = tf.keras.layers.experimental.preprocessing.RandomHeight(factor=0.25)
rand_width_module = tf.keras.layers.experimental.preprocessing.RandomWidth(factor=0.25)
contrast_module = tf.keras.layers.experimental.preprocessing.RandomContrast(factor=0.3)
rescale_module = tf.keras.layers.experimental.preprocessing.Rescaling(scale=1/255)
zoom_module = tf.keras.layers.experimental.preprocessing.RandomZoom(0.5, 0.3)

In [5]:
def load_model():
    base = VGG19(
        weights='imagenet',
        include_top=False,
        input_shape=(None, None, 3))

    x = base.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(5,
              activation='softmax')(x)
    m = Model(inputs=base.input, outputs=x)
    return m

In [6]:
_opt = tf.keras.optimizers.Adam(lr=1e-05)
_loss = CategoricalCrossentropy(label_smoothing=0.05)
_epochs = 20
_finetune_epochs = 10

model_ver_4 = load_model()

80142336/80134624 [==============================] - 1s 0us/step


In [7]:
model = tf.keras.Sequential([input_module, 
                             flip_module,
                             roration_module,
                             crop_module,
                             rand_height_module,
                             rand_width_module,
                             contrast_module,
                             zoom_module,
                             rescale_module,
                             model_ver_4])

In [8]:
model.compile(
        optimizer=_opt,
        loss=_loss,
        metrics=['accuracy']
    )

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
random_flip (RandomFlip)     (None, 512, 512, 3)       0         
_________________________________________________________________
random_rotation (RandomRotat (None, 512, 512, 3)       0         
_________________________________________________________________
random_crop (RandomCrop)     (None, 336, 336, 3)       0         
_________________________________________________________________
random_height (RandomHeight) (None, None, 336, 3)      0         
_________________________________________________________________
random_width (RandomWidth)   (None, None, None, 3)     0         
_________________________________________________________________
random_contrast (RandomContr (None, None, None, 3)     0         
_________________________________________________________________
random_zoom (RandomZoom)     (None, None, None, 3)     0

In [10]:
import pickle
_es = EarlyStopping(monitor='val_loss',
                    mode='min',
                    patience=10)
_r = ReduceLROnPlateau(monitor='val_loss',
                       factor=0.8,
                       patience=2,
                       verbose=1,
                       mode='auto',
                       epsilon=0.0001,
                       cooldown=5,
                       min_lr=0.00001)

_callback = [_es, _r]

history = model.fit(train_data, validation_data=valid_data, epochs=_epochs, callbacks=_callback, steps_per_epoch=_n_train//_batch_size)
model.save('/content/drive/MyDrive/cassava-leaf-disease-classification/VGG19_ver_1.h5')
with open('/content/drive/MyDrive/cassava-leaf-disease-classification/VGG19_ver_1.pkl', 'wb') as f:
    pickle.dump(history.history, f, pickle.HIGHEST_PROTOCOL)

Epoch 1/20
585/585 [==============================] - 2656s 4s/step - loss: 1.0503 - accuracy: 0.6423 - val_loss: 0.7782 - val_accuracy: 0.7657
Epoch 2/20
585/585 [==============================] - 2010s 3s/step - loss: 0.7726 - accuracy: 0.7622 - val_loss: 0.7166 - val_accuracy: 0.7997
Epoch 3/20
585/585 [==============================] - 1726s 3s/step - loss: 0.7056 - accuracy: 0.8000 - val_loss: 0.6413 - val_accuracy: 0.8281
Epoch 4/20
585/585 [==============================] - 1502s 3s/step - loss: 0.6872 - accuracy: 0.8041 - val_loss: 0.6367 - val_accuracy: 0.8330
Epoch 5/20
585/585 [==============================] - 1436s 2s/step - loss: 0.6659 - accuracy: 0.8133 - val_loss: 0.6036 - val_accuracy: 0.8460
Epoch 6/20
585/585 [==============================] - 1271s 2s/step - loss: 0.6431 - accuracy: 0.8256 - val_loss: 0.6004 - val_accuracy: 0.8501
Epoch 7/20
585/585 [==============================] - 1176s 2s/step - loss: 0.6393 - accuracy: 0.8273 - val_loss: 0.6149 - val_accuracy:

In [11]:
for i in model_ver_4.layers:
    i.trainable = True
    if 'bn' in i.name:
        i.trainable = False
history2 = model.fit(train_data, validation_data=valid_data, epochs=10, callbacks=_callback, steps_per_epoch=_n_train//_batch_size)
model.save('/content/drive/MyDrive/cassava-leaf-disease-classification/VGG19_ver_1_cp2.h5')
with open('/content/drive/MyDrive/cassava-leaf-disease-classification/VGG19_ver_1_cp2.pkl', 'wb') as f:
    pickle.dump(history.history, f, pickle.HIGHEST_PROTOCOL)

Epoch 1/10
585/585 [==============================] - 678s 1s/step - loss: 0.5681 - accuracy: 0.8572 - val_loss: 0.5676 - val_accuracy: 0.8580
Epoch 2/10
585/585 [==============================] - 669s 1s/step - loss: 0.5670 - accuracy: 0.8584 - val_loss: 0.5655 - val_accuracy: 0.8610
Epoch 3/10
585/585 [==============================] - 602s 1s/step - loss: 0.5593 - accuracy: 0.8624 - val_loss: 0.5548 - val_accuracy: 0.8655
Epoch 4/10
585/585 [==============================] - 632s 1s/step - loss: 0.5610 - accuracy: 0.8593 - val_loss: 0.5389 - val_accuracy: 0.8670
Epoch 5/10
585/585 [==============================] - 607s 1s/step - loss: 0.5576 - accuracy: 0.8612 - val_loss: 0.5854 - val_accuracy: 0.8524
Epoch 6/10
585/585 [==============================] - 604s 1s/step - loss: 0.5558 - accuracy: 0.8631 - val_loss: 0.5456 - val_accuracy: 0.8722
Epoch 7/10
585/585 [==============================] - 571s 977ms/step - loss: 0.5499 - accuracy: 0.8641 - val_loss: 0.5417 - val_accuracy: 0.8